<a href="https://colab.research.google.com/github/javaclll/glaciallake/blob/main/glaciallake_sentinel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mss

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
ee.Authenticate(auth_mode="colab")

In [ ]:
ee.Initialize(project='glaciallake')

In [ ]:
xMin = 86.7223
yMin = 27.9661
xMax = 86.8473
yMax = 27.9110

roi = ee.Geometry.Rectangle(
  [
    [xMin, yMin],
    [xMax, yMax]
  ]
);

In [ ]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

def clipper(image):
  return image.clip(roi)

landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
.filterDate('2019-01-01', '2022-01-02')\
.filterBounds(roi)

landsat = landsat.map(apply_scale_factors)
# landsat = landsat.map(clipper)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

Map = geemap.Map()
Map.add_layer(landsat, visualization, 'True Color (432)')
# Map.addLayer(roi, {}, 'roi')
Map.centerObject(roi, 12)
Map

Map(center=[27.938561768034454, 86.78480000000008], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask).divide(10000)

# Function to calculate NDWI
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']) # NIR (B8) and Green (B3) bands
    return ndwi.rename('NDWI')

# Load Sentinel-2 data and apply cloud masking
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-04-02', '2022-01-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# Calculate NDWI
ndwi_collection = dataset.map(calculate_ndwi)

# Visualization parameters
visualization_rgb = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B8', 'B4', 'B3'],
}

visualization_ndwi = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['008000', '00FF00', '0000FF'], # Green to blue palette
}

# Create a Map instance
m = geemap.Map()

# Set center of the map
m.set_center(86.7868, 27.9237, 12)

# Add RGB layer
m.add_layer(dataset.median(), visualization_rgb, 'RGB')

# Add NDWI layer
# m.add_layer(ndwi_collection.median(), visualization_ndwi, 'NDWI')

m

# aoi = ee.Geometry.Point(86.7868, 27.9237).buffer(5000)

# image = dataset.filterBounds(aoi).first()

# clipped_image = image.clip(roi)

# url = clipped_image.getThumbURL({'min': 0, 'max': 3000, 'dimensions': 1000})

# Image(url=url)


Map(center=[27.9237, 86.7868], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…